# Confflow Demo

## Imports

In [1]:
from typing import Literal, Optional

from confflow import BaseSchema, SchemaField, confflow

## Schema as Code

In [2]:
class CommonSettings(BaseSchema):
    name: str = SchemaField(..., min_length=3, max_length=50, description="The name of the configuration.")
    enabled: bool = SchemaField(default=True, description="Indicates if this configuration is enabled.")
    priority: int = SchemaField(default=1, ge=1, le=10, description="Priority level, must be between 1 and 10.")


class DatabaseConfig(BaseSchema):
    db_url: str = SchemaField(..., pattern=r"^(postgres|mysql|sqlite)://", description="Database connection URL.")
    max_connections: int = SchemaField(default=10, ge=1, le=100, description="Max number of connections.")
    timeout: Optional[int] = SchemaField(default=30, ge=10, le=300, description="Timeout in seconds.")


class APIConfig(BaseSchema):
    endpoint: str = SchemaField(..., pattern=r"^https?://", description="API endpoint URL.")
    auth_token: Optional[str] = SchemaField(None, description="Optional authentication token.")
    retries: int = SchemaField(default=3, ge=0, le=10, description="Number of retries in case of failure.")


class FeatureFlags(BaseSchema):
    experimental_feature: bool = SchemaField(default=False, description="Toggle for experimental feature.")
    legacy_mode: bool = SchemaField(default=False, description="Enable legacy mode for backward compatibility.")


# Mutually Exclusive Models
class FileStorageConfig(BaseSchema):
    storage_path: str = SchemaField(..., description="Path to the storage directory.")
    max_size_mb: int = SchemaField(default=100, ge=10, le=1024, description="Maximum storage size in MB.")
    backup_enabled: bool = SchemaField(default=True, description="Enable backup for stored files.")


class CloudStorageConfig(BaseSchema):
    provider: Literal['aws', 'gcp', 'azure'] = SchemaField(..., description="Cloud storage provider.")
    bucket_name: str = SchemaField(..., min_length=3, description="Name of the cloud storage bucket.")
    region: Optional[str] = SchemaField(None, description="Optional region of the cloud storage bucket.")

## Registrating Schemas

In [3]:
confflow.register_schemas(
    CommonSettings,
    FeatureFlags,
    CloudStorageConfig,
    DatabaseConfig
)

In [ ]:
# TODO mutual exlusion directly as list!
confflow.register_schemas(
    CommonSettings,
    FeatureFlags,
    [
        CloudStorageConfig,
        DatabaseConfig
    ]
)

In [4]:
confflow.set_mutual_exclusive_groups([CloudStorageConfig, DatabaseConfig])

In [ ]:
confflow.create_template("template_config.yml")

In [ ]:
confflow.load_yaml("config.yml")

In [ ]:
confflow["CommonSettings"]

In [ ]:
confflow["CommonSettings"]["enabled"]